# Small N Optimization (N=2-10)

Focus on small N values which have the highest per-tree score contribution.
Use exhaustive angle search and SA to find better configurations.

In [1]:
import numpy as np
import pandas as pd
import random
import math
import time
import json
from itertools import product
from shapely.geometry import Polygon
import matplotlib.pyplot as plt

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

print("Libraries loaded")

Libraries loaded


In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree."""
    def __init__(self, center_x=0.0, center_y=0.0, angle=0.0):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        self._update_polygon()
    
    def _update_polygon(self):
        angle_rad = np.radians(self.angle)
        cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
        rotated_x = TX * cos_a - TY * sin_a + self.center_x
        rotated_y = TX * sin_a + TY * cos_a + self.center_y
        self.polygon = Polygon(zip(rotated_x, rotated_y))
        self.vertices_x = rotated_x
        self.vertices_y = rotated_y
    
    def set_params(self, x, y, angle):
        self.center_x = float(x)
        self.center_y = float(y)
        self.angle = float(angle) % 360
        self._update_polygon()
    
    def clone(self):
        return ChristmasTree(self.center_x, self.center_y, self.angle)

def has_collision(trees):
    if len(trees) <= 1:
        return False
    for i in range(len(trees)):
        for j in range(i + 1, len(trees)):
            if trees[i].polygon.intersects(trees[j].polygon) and not trees[i].polygon.touches(trees[j].polygon):
                return True
    return False

def calculate_bounding_box(trees):
    all_x = np.concatenate([t.vertices_x for t in trees])
    all_y = np.concatenate([t.vertices_y for t in trees])
    return max(all_x.max() - all_x.min(), all_y.max() - all_y.min())

def calculate_score(trees):
    side = calculate_bounding_box(trees)
    return (side ** 2) / len(trees)

print("Helper functions defined")

Helper functions defined


In [3]:
# Load baseline
baseline_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
baseline_df = pd.read_csv(baseline_path)

def get_baseline_trees(n):
    prefix = f"{n:03d}_"
    trees_data = baseline_df[baseline_df['id'].str.startswith(prefix)]
    trees = []
    for _, row in trees_data.iterrows():
        x = float(str(row['x'])[1:])
        y = float(str(row['y'])[1:])
        deg = float(str(row['deg'])[1:])
        trees.append(ChristmasTree(x, y, deg))
    return trees

# Get baseline scores for small N
print("Baseline scores for small N:")
baseline_scores = {}
for n in range(1, 11):
    trees = get_baseline_trees(n)
    score = calculate_score(trees)
    baseline_scores[n] = score
    print(f"N={n}: score={score:.6f}, side={np.sqrt(score * n):.6f}")

Baseline scores for small N:
N=1: score=0.661250, side=0.813173
N=2: score=0.450779, side=0.949504
N=3: score=0.434745, side=1.142031
N=4: score=0.416545, side=1.290806
N=5: score=0.416850, side=1.443692
N=6: score=0.399610, side=1.548438
N=7: score=0.399897, side=1.673104
N=8: score=0.385407, side=1.755921
N=9: score=0.387415, side=1.867280
N=10: score=0.376630, side=1.940696


In [4]:
# Simulated Annealing for small N
class SmallNOptimizer:
    def __init__(self, n, Tmax=0.1, Tmin=0.00001, nsteps=20000, random_state=42):
        self.n = n
        self.Tmax = Tmax
        self.Tmin = Tmin
        self.nsteps = nsteps
        random.seed(random_state)
        np.random.seed(random_state)
        
        # Initialize from baseline
        self.trees = get_baseline_trees(n)
        self.best_trees = [t.clone() for t in self.trees]
        self.best_score = calculate_score(self.trees)
    
    def perturb(self, tree_idx, position_delta=0.02, angle_delta=2.0):
        tree = self.trees[tree_idx]
        old_params = (tree.center_x, tree.center_y, tree.angle)
        
        choice = random.randint(0, 2)
        if choice == 0:
            tree.center_x += random.uniform(-position_delta, position_delta)
        elif choice == 1:
            tree.center_y += random.uniform(-position_delta, position_delta)
        else:
            tree.angle = (tree.angle + random.uniform(-angle_delta, angle_delta)) % 360
        tree._update_polygon()
        
        return old_params
    
    def rollback(self, tree_idx, old_params):
        self.trees[tree_idx].set_params(*old_params)
    
    def solve(self, verbose=True):
        current_score = self.best_score
        T = self.Tmax
        cooling_rate = (self.Tmin / self.Tmax) ** (1.0 / self.nsteps)
        
        for step in range(self.nsteps):
            tree_idx = random.randint(0, self.n - 1)
            old_params = self.perturb(tree_idx)
            
            if has_collision(self.trees):
                self.rollback(tree_idx, old_params)
                continue
            
            new_score = calculate_score(self.trees)
            delta = new_score - current_score
            
            if delta < 0 or random.random() < math.exp(-delta / T):
                current_score = new_score
                if new_score < self.best_score:
                    self.best_score = new_score
                    self.best_trees = [t.clone() for t in self.trees]
            else:
                self.rollback(tree_idx, old_params)
            
            T *= cooling_rate
            
            if verbose and step % 5000 == 0:
                print(f"Step {step}, T={T:.8f}, Best={self.best_score:.6f}")
        
        return self.best_score, self.best_trees

print("SmallNOptimizer defined")

SmallNOptimizer defined


In [5]:
# Optimize small N values
results = {}

for n in range(2, 11):
    print(f"\n{'='*50}")
    print(f"Optimizing N={n}...")
    
    baseline_score = baseline_scores[n]
    print(f"Baseline score: {baseline_score:.6f}")
    
    # Run multiple random seeds
    best_score = baseline_score
    best_trees = None
    
    for seed in [42, 123, 456, 789, 1000]:
        optimizer = SmallNOptimizer(n, Tmax=0.05, Tmin=0.000001, nsteps=15000, random_state=seed)
        score, trees = optimizer.solve(verbose=False)
        
        if score < best_score:
            best_score = score
            best_trees = trees
            print(f"  Seed {seed}: NEW BEST {score:.6f}")
        else:
            print(f"  Seed {seed}: {score:.6f}")
    
    improvement = baseline_score - best_score
    results[n] = {
        'baseline': baseline_score,
        'optimized': best_score,
        'improvement': improvement,
        'trees': best_trees
    }
    print(f"Final: {best_score:.6f}, Improvement: {improvement:.6f}")


Optimizing N=2...
Baseline score: 0.450779


  Seed 42: 0.450779


  Seed 123: 0.450779


  Seed 456: 0.450779


  Seed 789: 0.450779


...

  Seed 42: 0.376630


  Seed 123: 0.376630


  Seed 456: 0.376630


  Seed 789: 0.376630


  Seed 1000: 0.376630
Final: 0.376630, Improvement: 0.000000


In [6]:
# The baseline is already at local optimum for small N\n# Let's try generating NEW configurations from scratch\n# with different initial placements\n\nclass RandomStartOptimizer:\n    \"\"\"Generate random initial configurations and optimize.\"\"\"\n    def __init__(self, n, Tmax=0.5, Tmin=0.00001, nsteps=30000, random_state=42):\n        self.n = n\n        self.Tmax = Tmax\n        self.Tmin = Tmin\n        self.nsteps = nsteps\n        random.seed(random_state)\n        np.random.seed(random_state)\n        \n        # Generate random initial configuration\n        self.trees = self._generate_random_config()\n        self.best_trees = [t.clone() for t in self.trees]\n        self.best_score = float('inf') if has_collision(self.trees) else calculate_score(self.trees)\n    \n    def _generate_random_config(self):\n        \"\"\"Generate a random collision-free configuration.\"\"\"\n        trees = []\n        max_attempts = 1000\n        \n        for i in range(self.n):\n            for attempt in range(max_attempts):\n                # Random position in a reasonable range\n                x = random.uniform(-2, 2)\n                y = random.uniform(-2, 2)\n                angle = random.uniform(0, 360)\n                \n                new_tree = ChristmasTree(x, y, angle)\n                \n                # Check collision with existing trees\n                collision = False\n                for t in trees:\n                    if new_tree.polygon.intersects(t.polygon) and not new_tree.polygon.touches(t.polygon):\n                        collision = True\n                        break\n                \n                if not collision:\n                    trees.append(new_tree)\n                    break\n            else:\n                # If we couldn't place, just add it anyway (SA will fix it)\n                trees.append(ChristmasTree(random.uniform(-2, 2), random.uniform(-2, 2), random.uniform(0, 360)))\n        \n        return trees\n    \n    def perturb(self, tree_idx, position_delta=0.05, angle_delta=5.0):\n        tree = self.trees[tree_idx]\n        old_params = (tree.center_x, tree.center_y, tree.angle)\n        \n        choice = random.randint(0, 2)\n        if choice == 0:\n            tree.center_x += random.uniform(-position_delta, position_delta)\n        elif choice == 1:\n            tree.center_y += random.uniform(-position_delta, position_delta)\n        else:\n            tree.angle = (tree.angle + random.uniform(-angle_delta, angle_delta)) % 360\n        tree._update_polygon()\n        \n        return old_params\n    \n    def rollback(self, tree_idx, old_params):\n        self.trees[tree_idx].set_params(*old_params)\n    \n    def solve(self, verbose=False):\n        current_score = self.best_score\n        T = self.Tmax\n        cooling_rate = (self.Tmin / self.Tmax) ** (1.0 / self.nsteps)\n        \n        for step in range(self.nsteps):\n            tree_idx = random.randint(0, self.n - 1)\n            old_params = self.perturb(tree_idx)\n            \n            if has_collision(self.trees):\n                new_score = float('inf')\n            else:\n                new_score = calculate_score(self.trees)\n            \n            delta = new_score - current_score\n            \n            if delta < 0 or (new_score < float('inf') and random.random() < math.exp(-delta / T)):\n                current_score = new_score\n                if new_score < self.best_score:\n                    self.best_score = new_score\n                    self.best_trees = [t.clone() for t in self.trees]\n            else:\n                self.rollback(tree_idx, old_params)\n            \n            T *= cooling_rate\n        \n        return self.best_score, self.best_trees\n\nprint(\"RandomStartOptimizer defined\")"}, {"cell_type": "code", "content": "# Try random start optimization for small N\nprint(\"Trying random start optimization...\")\n\nrandom_results = {}\n\nfor n in range(2, 6):  # Focus on smallest N first\n    print(f\"\\n{'='*50}\")\n    print(f\"Random start optimization for N={n}...\")\n    \n    baseline_score = baseline_scores[n]\n    print(f\"Baseline score: {baseline_score:.6f}\")\n    \n    best_score = baseline_score\n    best_trees = None\n    \n    # Try many random starts\n    for seed in range(50):\n        optimizer = RandomStartOptimizer(n, Tmax=0.3, Tmin=0.000001, nsteps=20000, random_state=seed)\n        score, trees = optimizer.solve(verbose=False)\n        \n        if score < best_score:\n            best_score = score\n            best_trees = trees\n            print(f\"  Seed {seed}: NEW BEST {score:.6f}\")\n    \n    improvement = baseline_score - best_score\n    random_results[n] = {\n        'baseline': baseline_score,\n        'optimized': best_score,\n        'improvement': improvement,\n        'trees': best_trees\n    }\n    print(f\"Final: {best_score:.6f}, Improvement: {improvement:.6f}\")"}, {"cell_type": "code", "content": "# Summary of random start results\nprint(\"\\n\" + \"=\"*60)\nprint(\"SUMMARY OF RANDOM START OPTIMIZATION\")\nprint(\"=\"*60)\n\ntotal_improvement = 0\nfor n in sorted(random_results.keys()):\n    r = random_results[n]\n    status = \"\u2713 IMPROVED\" if r['improvement'] > 0 else \"\u2717 No improvement\"\n    print(f\"N={n}: Baseline={r['baseline']:.6f}, Optimized={r['optimized']:.6f}, \u0394={r['improvement']:+.6f} {status}\")\n    total_improvement += r['improvement']\n\nprint(f\"\\nTotal improvement from random start: {total_improvement:.6f}\")"}]

In [7]:
# Summary
print("\n" + "="*60)
print("SUMMARY OF SMALL N OPTIMIZATION")
print("="*60)

total_improvement = 0
for n in range(2, 11):
    r = results[n]
    status = "✓ IMPROVED" if r['improvement'] > 0 else "✗ No improvement"
    print(f"N={n}: Baseline={r['baseline']:.6f}, Optimized={r['optimized']:.6f}, Δ={r['improvement']:+.6f} {status}")
    total_improvement += r['improvement']

print(f"\nTotal improvement from small N: {total_improvement:.6f}")


SUMMARY OF SMALL N OPTIMIZATION
N=2: Baseline=0.450779, Optimized=0.450779, Δ=+0.000000 ✗ No improvement
N=3: Baseline=0.434745, Optimized=0.434745, Δ=+0.000000 ✗ No improvement
N=4: Baseline=0.416545, Optimized=0.416545, Δ=+0.000000 ✗ No improvement
N=5: Baseline=0.416850, Optimized=0.416850, Δ=+0.000000 ✗ No improvement
N=6: Baseline=0.399610, Optimized=0.399610, Δ=+0.000000 ✗ No improvement
N=7: Baseline=0.399897, Optimized=0.399897, Δ=+0.000000 ✗ No improvement
N=8: Baseline=0.385407, Optimized=0.385407, Δ=+0.000000 ✗ No improvement
N=9: Baseline=0.387415, Optimized=0.387415, Δ=+0.000000 ✗ No improvement
N=10: Baseline=0.376630, Optimized=0.376630, Δ=+0.000000 ✗ No improvement

Total improvement from small N: 0.000000


In [8]:
# Try more aggressive optimization for N=2 (highest per-tree impact after N=1)
print("\nAggressive optimization for N=2...")

best_score_n2 = baseline_scores[2]
best_trees_n2 = None

for seed in range(100):  # Try 100 different seeds
    optimizer = SmallNOptimizer(2, Tmax=0.1, Tmin=0.0000001, nsteps=30000, random_state=seed)
    score, trees = optimizer.solve(verbose=False)
    
    if score < best_score_n2:
        best_score_n2 = score
        best_trees_n2 = trees
        print(f"Seed {seed}: NEW BEST {score:.6f}")

print(f"\nBest N=2 score: {best_score_n2:.6f}")
print(f"Improvement: {baseline_scores[2] - best_score_n2:.6f}")


Aggressive optimization for N=2...



Best N=2 score: 0.450779
Improvement: 0.000000


In [ ]:
# Create submission with improvements
print("\nCreating submission with improvements...")

# Load baseline submission
submission_df = baseline_df.copy()

# Function to format value with 's' prefix
def format_val(v):
    return f"s{v}"

# Update improved N values
for n, r in results.items():
    if r['improvement'] > 0 and r['trees'] is not None:
        prefix = f"{n:03d}_"
        # Remove old rows for this N
        submission_df = submission_df[~submission_df['id'].str.startswith(prefix)]
        
        # Add new rows
        new_rows = []
        for i, tree in enumerate(r['trees']):
            new_rows.append({
                'id': f"{n:03d}_{i}",
                'x': format_val(tree.center_x),
                'y': format_val(tree.center_y),
                'deg': format_val(tree.angle)
            })
        submission_df = pd.concat([submission_df, pd.DataFrame(new_rows)], ignore_index=True)

# Sort by id
submission_df['sort_key'] = submission_df['id'].apply(lambda x: (int(x.split('_')[0]), int(x.split('_')[1])))
submission_df = submission_df.sort_values('sort_key').drop('sort_key', axis=1)

print(f"Submission has {len(submission_df)} rows")

In [ ]:
# Calculate total score of new submission
def calculate_total_score(df):
    total = 0
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        trees_data = df[df['id'].str.startswith(prefix)]
        trees = []
        for _, row in trees_data.iterrows():
            x = float(str(row['x'])[1:]) if str(row['x']).startswith('s') else float(row['x'])
            y = float(str(row['y'])[1:]) if str(row['y']).startswith('s') else float(row['y'])
            deg = float(str(row['deg'])[1:]) if str(row['deg']).startswith('s') else float(row['deg'])
            trees.append(ChristmasTree(x, y, deg))
        if len(trees) == n:
            total += calculate_score(trees)
    return total

new_total_score = calculate_total_score(submission_df)
baseline_total_score = calculate_total_score(baseline_df)

print(f"Baseline total score: {baseline_total_score:.6f}")
print(f"New total score: {new_total_score:.6f}")
print(f"Improvement: {baseline_total_score - new_total_score:.6f}")

In [ ]:
# Save submission
import os
os.makedirs('/home/submission', exist_ok=True)
submission_df.to_csv('/home/submission/submission.csv', index=False)
print("Saved submission to /home/submission/submission.csv")

# Save metrics
metrics = {
    'cv_score': new_total_score,
    'baseline_score': baseline_total_score,
    'improvement': baseline_total_score - new_total_score,
    'small_n_improvements': {str(n): r['improvement'] for n, r in results.items()}
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nFinal CV Score: {new_total_score:.6f}")
print(f"Target: 68.919154")
print(f"Gap: {new_total_score - 68.919154:.6f}")

In [9]:
# Run random start optimization\nprint(\"Running random start optimization...\")\n\nrandom_results = {}\n\nfor n in range(2, 6):  # Focus on smallest N first\n    print(f\"\\n{'='*50}\")\n    print(f\"Random start optimization for N={n}...\")\n    \n    baseline_score = baseline_scores[n]\n    print(f\"Baseline score: {baseline_score:.6f}\")\n    \n    best_score = baseline_score\n    best_trees = None\n    \n    # Try many random starts\n    for seed in range(30):\n        optimizer = RandomStartOptimizer(n, Tmax=0.3, Tmin=0.000001, nsteps=15000, random_state=seed)\n        score, trees = optimizer.solve(verbose=False)\n        \n        if score < best_score:\n            best_score = score\n            best_trees = trees\n            print(f\"  Seed {seed}: NEW BEST {score:.6f}\")\n    \n    improvement = baseline_score - best_score\n    random_results[n] = {\n        'baseline': baseline_score,\n        'optimized': best_score,\n        'improvement': improvement,\n        'trees': best_trees\n    }\n    print(f\"Final: {best_score:.6f}, Improvement: {improvement:.6f}\")\n\nprint(\"\\nRandom start optimization complete.\")

In [ ]:
# Summary of random start results\nprint(\"\\n\" + \"=\"*60)\nprint(\"SUMMARY OF RANDOM START OPTIMIZATION\")\nprint(\"=\"*60)\n\ntotal_improvement = 0\nfor n in sorted(random_results.keys()):\n    r = random_results[n]\n    status = \"\u2713 IMPROVED\" if r['improvement'] > 0 else \"\u2717 No improvement\"\n    print(f\"N={n}: Baseline={r['baseline']:.6f}, Optimized={r['optimized']:.6f}, \u0394={r['improvement']:+.6f} {status}\")\n    total_improvement += r['improvement']\n\nprint(f\"\\nTotal improvement from random start: {total_improvement:.6f}\")"}